In [39]:
import torch
import numpy as np
import shap
from PIL import Image
from torchvision import transforms
from torchvision.models import vgg16

model = vgg16(pretrained=True)
model.eval()

input_size = 112

# Create a transform that resizes the image to the required input size and normalizes the pixel values
preprocess = transforms.Compose([
    transforms.Resize(input_size),
    transforms.CenterCrop(input_size),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Preprocess the image tensor
preprocessed_image = preprocess(Image.open("/Users/faizahkureshi/Desktop/Capstone Project/dataset/test/Elephant/ele.png"))

# Convert the preprocessed image tensor to a tensor that can be used as input to the model
tensor_image = preprocessed_image.unsqueeze(0)

# Compute SHAP values using the tensor_image tensor
shap_values = explainer.shap_values(tensor_image)

RuntimeError: The size of tensor a (4) must match the size of tensor b (3) at non-singleton dimension 0

In [41]:
import json

import numpy as np
import torch
from torchvision import models

import shap

In [42]:
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]


def normalize(image):
    if image.max() > 1:
        image /= 255
    image = (image - mean) / std
    # in addition, roll the axis so that they suit pytorch
    return torch.tensor(image.swapaxes(-1, 1).swapaxes(2, 3)).float()

In [62]:
# load the model
model = models.vgg16(pretrained=True).eval()

X, y = shap.datasets.imagenet50()

X /= 255

to_explain = X[[40, 41]]

# load the ImageNet class names
url = "https://s3.amazonaws.com/deep-learning-models/image-models/imagenet_class_index.json"
fname = shap.datasets.cache(url)
with open(fname) as f:
    class_names = json.load(f)

In [64]:
# note that because the inputs are scaled to be between 0 and 1, the local smoothing also has to be
# scaled compared to the Keras model
explainer = shap.GradientExplainer(
    (model, model.features[7]), normalize(X), local_smoothing=0.5
)
shap_values, indexes = explainer.shap_values(
    normalize(to_explain), ranked_outputs=2, nsamples=10
)

# get the names for the classes
index_names = np.vectorize(lambda x: class_names[str(x)][1])(indexes)

# plot the explanations
shap_values = [np.swapaxes(np.swapaxes(s, 2, 3), 1, -1) for s in shap_values]


In [65]:
print("Shape of SHAP values array:", [s.shape for s in shap_values])

Shape of SHAP values array: [(128, 112, 2, 112), (128, 112, 2, 112)]


In [66]:
print("Shape of input data:", to_explain.shape)

Shape of input data: (2, 224, 224, 3)


In [ ]:

shap.image_plot(shap_values, to_explain, index_names)

In [68]:
index_names = index_names.reshape((len(to_explain), -1))
print(index_names)

[['beer_bottle' 'beer_glass']
 ['meerkat' 'mongoose']]


In [76]:
import torch
import torchvision.transforms as transforms
from PIL import Image
import numpy as np
import shap
import json
import matplotlib.pyplot as plt

mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

# Load the model
model = models.vgg16(pretrained=True).eval()

# Load the ImageNet class names
url = "https://s3.amazonaws.com/deep-learning-models/image-models/imagenet_class_index.json"
fname = shap.datasets.cache(url)
with open(fname) as f:
    class_names = json.load(f)

# Load and preprocess the input image
input_size = 224
preprocess = transforms.Compose([
    transforms.Resize(input_size),
    transforms.CenterCrop(input_size),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std),
])

# Load and preprocess the image tensor
image_path = "/Users/faizahkureshi/Desktop/Capstone Project/dataset/test/Elephant/ele.png"
image = Image.open(image_path)
preprocessed_image = preprocess(image)

# Ensure the input tensor has the correct shape (1, 3, height, width)
preprocessed_image = preprocessed_image.unsqueeze(0)

# Get SHAP values
explainer = shap.GradientExplainer((model, model.features[7]), torch.zeros(1, 3, input_size, input_size))
shap_values, indexes = explainer.shap_values(preprocessed_image, ranked_outputs=2, nsamples=10)

# Rescale SHAP values to match input image dimensions
shap_values = [s[0].detach().cpu().numpy() for s in shap_values]
shap_values = [s.transpose(1, 2, 0) for s in shap_values]

# Plot SHAP values overlayed on the input image
plt.figure()
plt.imshow(preprocessed_image.squeeze(0).permute(1, 2, 0))  # Convert tensor to numpy array and permute dimensions
for i in range(len(shap_values)):
    plt.imshow(shap_values[i].sum(axis=-1), cmap='jet', alpha=0.5)
plt.colorbar()
plt.show()


RuntimeError: The size of tensor a (4) must match the size of tensor b (3) at non-singleton dimension 0